
# SFT with Template Masking (LoRA) — Argilla/Bitext + Dolly (7B)

This notebook fine-tunes a chat model (e.g., `meta-llama/Llama-2-7b-hf`) using **template masking** (no collator), so only the **assistant** parts are trained.  
Key points:

- **No DataCollator** required. We use the tokenizer's **chat template** with `{% generation %}...{% endgeneration %}` to mask assistant turns.
- `packing=True` works (higher throughput) since masking is handled by the chat template.
- `tokenizer.truncation_side="left"` to preserve the assistant turn at the end of long sequences.
- Supports Argilla/Bitext customer-support datasets + Dolly-15k subset.
- Uses **LoRA** (PEFT) to reduce VRAM usage.


In [1]:

# %%capture
# # (Optional) Install versions known to work well together
!pip install -U transformers accelerate datasets peft bitsandbytes hf_transfer
!pip install -U trl>=0.25.0
# # (Optional) flash-attn (environment dependent)
# !pip install flash-attn --no-build-isolation


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 126.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 57.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 133.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 136.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 166.5 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 154.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 177.2 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.5/803.5 kB 83.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 190.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25/25 [datasets]/25 [datasets]e]s]ub]


In [2]:

from typing import Dict, Any, List
from datasets import load_dataset, DatasetDict, concatenate_datasets
from trl import SFTConfig, SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM

import random
import os

SEED = 42
random.seed(SEED)


In [21]:

# --- Hugging Face login helper ---
try:
    from huggingface_hub import login, whoami
    import os
    if os.environ.get("HF_TOKEN"):
        login(token=os.environ["HF_TOKEN"])
        try:
            print("HF whoami:", whoami())
        except Exception:
            pass
    else:
        print(" Set HF_TOKEN env var or run:")
        print("   from huggingface_hub import login; login(token='hf_...')")
except Exception as e:
    print("Hugging Face login not available:", e)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


HF whoami: {'type': 'user', 'id': '6900911d2cafc5f572673a1c', 'name': 'survd0404', 'fullname': 'Lee', 'isPro': False, 'avatarUrl': '/avatars/6738b5fc2f71a66ab3ca028ab9a5da26.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'HF_TOKEN', 'role': 'fineGrained', 'createdAt': '2025-11-05T14:30:18.423Z', 'fineGrained': {'canReadGatedRepos': True, 'global': ['discussion.write', 'post.write'], 'scoped': [{'entity': {'_id': '6900911d2cafc5f572673a1c', 'type': 'user', 'name': 'survd0404'}, 'permissions': ['repo.content.read', 'repo.write', 'inference.serverless.write', 'inference.endpoints.infer.write', 'inference.endpoints.write', 'user.webhooks.read', 'user.webhooks.write', 'collection.read', 'collection.write', 'discussion.write']}]}}}}


In [7]:

# Choose your base model (7B chat model recommended)
MODEL_NAME = os.environ.get("MODEL_NAME", "meta-llama/Llama-2-7b-hf")
# You must have access to the model on HF. If needed: `huggingface-cli login`

print("Loading tokenizer:", MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
# Truncation/padding so that assistant turn (at the end) is kept
tokenizer.truncation_side = "left"
tokenizer.padding_side    = "right"
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("Loading model:", MODEL_NAME)
# Use bf16 if on multi-GPU auto mapping; fp16 if single 24GB GPU for lower VRAM
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype="auto",     # change to torch.bfloat16 / torch.float16 explicitly if desired
    device_map="auto"       # let HF place layers automatically if multiple GPUs
)
# Disable cache during training
if hasattr(model, "config"):
    model.config.use_cache = False

print("Model & tokenizer ready.")


Loading tokenizer: meta-llama/Llama-2-7b-hf


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Model & tokenizer ready.


In [8]:

# Define a chat template that masks assistant turns only.
# With this, SFTTrainer will compute loss only on `{% generation %}` blocks.
tokenizer.chat_template = """{% for message in messages %}
<|{{ message['role'] }}|>
{% if message['role'] == 'assistant' -%}
{% generation %}
{{ message['content'] }}
{% endgeneration %}
{%- else -%}
{{ message['content'] }}
{%- endif %}

{% endfor %}"""

print("Chat template with assistant masking is set.")


Chat template with assistant masking is set.


In [9]:

# Datasets to pull
ARGILLA_BITEXT = [
    ("argilla/customer_assistant", {}),
    ("argilla/synthetic-sft-customer-support-single-turn", {}),
    ("bitext/Bitext-customer-support-llm-chatbot-training-dataset", {}),
]
DOLLY_SLICE = "train[:1000]"  # Adjust as needed

def extract_user_assistant(record: Dict[str, Any]) -> Dict[str, str]:
    """Map heterogeneous keys to a unified (system, user, assistant) triple."""
    keys = set(record.keys())

    user_candidates = [
        "user","question","prompt","instruction","input","request","query",
        "messages_user","customer","customer_message"
    ]
    assistant_candidates = [
        "assistant","answer","response","output","messages_assistant",
        "agent","agent_response"
    ]
    system_candidates = ["system","context","role","scenario"]

    def pick(cands):
        # 1) flat keys
        for c in cands:
            if c in record and isinstance(record[c], str) and len(record[c].strip())>0:
                return record[c]
        # 2) nested messages
        for k in keys:
            v = record[k]
            if isinstance(v, list) and len(v)>0 and isinstance(v[0], dict) and "role" in v[0] and "content" in v[0]:
                u, a, s = None, None, None
                for m in v:
                    role = (m.get("role","") or "").lower()
                    content = m.get("content","")
                    if role == "system" and not s:
                        s = content
                    if role in ("user","customer") and not u:
                        u = content
                    if role in ("assistant","agent") and not a:
                        a = content
                if cands is user_candidates and u:
                    return u
                if cands is assistant_candidates and a:
                    return a
                if cands is system_candidates and s:
                    return s
        return None

    user = pick(user_candidates)
    assistant = pick(assistant_candidates)
    system = pick(system_candidates) or "You are a helpful, step-by-step customer support assistant."

    # fallback: synthesize from any two text fields
    if not user or not assistant:
        textish = [k for k in keys if isinstance(record.get(k), str) and len(record[k].strip())>0]
        if len(textish)>=2 and not user:
            user = record[textish[0]]
        if len(textish)>=2 and not assistant:
            assistant = record[textish[1]]

    if not user or not assistant:
        raise ValueError("Could not map record to user/assistant")

    return {"system": system.strip(), "user": user.strip(), "assistant": assistant.strip()}


In [10]:

def preprocess_to_messages(hf_id: str, hf_config: Dict[str, Any]):
    ds = load_dataset(hf_id, **hf_config)
    split_names = list(ds.keys()) if isinstance(ds, DatasetDict) else ["train"]
    out_splits = []

    for split in split_names:
        d = ds[split] if isinstance(ds, DatasetDict) else ds
        def _map(rec):
            try:
                ex = extract_user_assistant(rec)
                msgs = []
                if ex.get("system"):
                    msgs.append({"role":"system","content":ex["system"]})
                msgs.append({"role":"user","content":ex["user"]})
                msgs.append({"role":"assistant","content":ex["assistant"]})
                return {"messages": msgs}
            except Exception:
                return {"messages": None}

        d2 = d.map(_map, remove_columns=[c for c in d.column_names if c!="messages"])
        d2 = d2.filter(lambda r: r["messages"] is not None)
        out_splits.append(d2)

    merged = concatenate_datasets(out_splits) if len(out_splits)>1 else out_splits[0]
    return merged

def build_argilla_bitext_messages():
    all_ds = []
    for name, cfg in ARGILLA_BITEXT:
        print("Loading:", name, cfg)
        try:
            all_ds.append(preprocess_to_messages(name, cfg))
        except Exception as e:
            print("!! Skipped", name, "due to", e)
    if not all_ds:
        raise RuntimeError("No argilla/bitext datasets loaded — check access/names.")
    merged = concatenate_datasets(all_ds)
    return merged


In [11]:

def build_dolly_messages(slice_spec=DOLLY_SLICE):
    ds = load_dataset("databricks/databricks-dolly-15k", split=slice_spec)
    def _fmt(sample):
        instruction = sample.get("instruction","") or ""
        context = sample.get("context","") or ""
        response = sample.get("response","") or ""
        user = f"{instruction.strip()} {context.strip()}".strip()
        assistant = response.strip()
        msgs = []
        # Dolly usually has no system by default
        if user:
            msgs.append({"role":"user","content":user})
        if assistant:
            msgs.append({"role":"assistant","content":assistant})
        return {"messages": msgs if msgs else None}

    ds = ds.map(_fmt, remove_columns=[c for c in ds.column_names if c!="messages"])
    ds = ds.filter(lambda r: r["messages"] is not None)
    return ds


In [12]:

argilla_bitext = build_argilla_bitext_messages()
dolly_subset   = build_dolly_messages(DOLLY_SLICE)

dataset = concatenate_datasets([argilla_bitext, dolly_subset]).shuffle(seed=SEED)
print("Merged size (raw):", len(dataset))

def has_assistant(ex):
    m = ex.get("messages", [])
    roles = [x.get("role") for x in m]
    return isinstance(m, list) and ("assistant" in roles)

def min_tokens_chat(ex, min_tokens=8):
    s = tokenizer.apply_chat_template(ex["messages"], tokenize=False, add_generation_prompt=False)
    ids = tokenizer(s, add_special_tokens=False).input_ids
    return ids is not None and len(ids) >= min_tokens

dataset = dataset.filter(has_assistant)
dataset = dataset.filter(min_tokens_chat)
print("After basic filtering:", len(dataset))

# Quick peek
for i in range(2):
    s = tokenizer.apply_chat_template(dataset[i]["messages"], tokenize=False, add_generation_prompt=False)
    print("----", s[:400])


Loading: argilla/customer_assistant {}


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-6853b30b1d9b88(…):   0%|          | 0.00/76.1k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/196 [00:00<?, ? examples/s]

Map:   0%|          | 0/196 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

Merged size (raw): 28168


Filter:   0%|          | 0/28168 [00:00<?, ? examples/s]

Filter:   0%|          | 0/28168 [00:00<?, ? examples/s]

After basic filtering: 28168
---- <|system|>
You are a helpful, step-by-step customer support assistant.
<|user|>
I want help seeing in which cases can I ask for a refund
<|assistant|>
Of course, I'm here to provide you with a detailed understanding of the cases in which you can request a refund. Here are some common scenarios:

1. **Product/Service Defect:** If the product or service you purchased is defective, damaged, or doesn'
---- <|system|>
You are a helpful, step-by-step customer support assistant.
<|user|>
i have a problem setting a delivery address up
<|assistant|>
I apologize for the difficulties you're experiencing while setting up your delivery address. I'm here to assist you in resolving this issue.

Setting up a delivery address is an essential part of the process, and I completely understand the importance of gett


In [13]:

n = len(dataset)
k = min(100, max(1, n // 50))  # about 2% or cap at 100
split = dataset.train_test_split(test_size=k, seed=SEED, shuffle=True)
train_dataset, eval_dataset = split["train"], split["test"]

print("train:", len(train_dataset), "eval:", len(eval_dataset))


train: 28068 eval: 100


In [14]:

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj","v_proj"],  # add k_proj/o_proj if needed
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
print("LoRA config ready.")


LoRA config ready.


In [15]:

sft_args = SFTConfig(
    output_dir="./sft_custom_results",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=1e-5,
    logging_steps=20,
    save_steps=200,
    save_total_limit=2,
    optim="adamw_torch",
    fp16=True,                 # On multi-GPU auto mapping, bf16=True can be safer
    packing=True,              # ✅ works with template masking
    max_length=512,
    eval_strategy="steps",
    eval_steps=100,
    load_best_model_at_end=True,
    gradient_checkpointing=True,   # big VRAM saver
)
print(sft_args)


SFTConfig(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
activation_offloading=False,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
assistant_only_loss=False,
auto_find_batch_size=False,
average_tokens_across_devices=True,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
chat_template_path=None,
completion_only_loss=None,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
dataset_kwargs=None,
dataset_num_proc=None,
dataset_text_field=text,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eos_token

In [16]:

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    processing_class=tokenizer,
    args=sft_args,
    peft_config=peft_config,
    formatting_func=None,
)
print("Trainer ready. Rendering one sample after template:")
print(tokenizer.apply_chat_template(train_dataset[0]["messages"], tokenize=False, add_generation_prompt=False)[:600])


Padding-free training is enabled, but the attention implementation is not set to a supported flash attention variant. Padding-free training flattens batches into a single sequence, and only the following implementations are known to reliably support this: flash_attention_2, flash_attention_3, kernels-community/flash-attn, kernels-community/flash-attn3, kernels-community/vllm-flash-attn3. Using other implementations may lead to unexpected behavior. To ensure compatibility, set `attn_implementation` in the model configuration to one of these supported options or verify that your attention mechanism can handle flattened sequences.
You are using packing, but the attention implementation is not set to a supported flash attention variant. Packing gathers multiple samples into a single sequence, and only the following implementations are known to reliably support this: flash_attention_2, flash_attention_3, kernels-community/flash-attn, kernels-community/flash-attn3, kernels-community/vllm-fla

Tokenizing train dataset:   0%|          | 0/28068 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/28068 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Packing eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


Trainer ready. Rendering one sample after template:
<|system|>
You are a helpful, step-by-step customer support assistant.
<|user|>
i have to see the status of purchase {{Order Number}} how do i do it
<|assistant|>
Glad you contacted us! I'm clearly cognizant that you would like to check the status of your purchase with order number {{Order Number}}. To do so, you can navigate to the 'Order Details' section on our website. This section will provide you with all the information regarding your purchase, including its current status. If you have any further questions or need additional assistance, please don't hesitate to ask. I'm here to help!




In [17]:

# 🚀 Train
train_result = trainer.train()
print("Training complete.")
print(train_result)


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.


Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
100,1.414800,1.361153,1.464559,799604.000000,0.662065
200,0.999400,1.030468,1.017812,1599955.000000,0.729319
300,0.914500,0.969202,0.962321,2398635.000000,0.740033
400,0.904600,0.930884,0.927350,3197100.000000,0.747152
500,0.870700,0.907071,0.904815,3997432.000000,0.751359
600,0.880100,0.895056,0.897542,4798588.000000,0.754803


Training complete.
TrainOutput(global_step=670, training_loss=1.0405721821002105, metrics={'train_runtime': 1830.4509, 'train_samples_per_second': 5.855, 'train_steps_per_second': 0.366, 'total_flos': 2.1264560405694874e+17, 'train_loss': 1.0405721821002105, 'epoch': 1.0})


In [18]:

# Save LoRA adapter
trainer.save_model()  # saves to output_dir

# Quick test generation
from transformers import TextStreamer

messages = [
    {"role":"system","content":"You are a helpful customer support assistant."},
    {"role":"user","content":"How can I track my order?"},
]
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
gen = model.generate(
    **inputs,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    streamer=streamer
)



<|assistant|>
To track your order, please provide me with the following information:

1. Your order number: {{Order Number}}
2. The shipping or tracking number associated with your order: {{Shipping or Tracking Number}}

Once I have these details, I will be able to provide you with the most up-to-date information on the status of your order. If you have any further questions or concerns, please feel free to reach out to us. We're here to help!




















































































































































## 🔍 모델 평가 (G-EVAL + RAG + 벤치마크 훅)

이 섹션에서는 다음을 수행할 수 있도록 평가 코드를 추가했습니다.

1. **G-EVAL 스타일 항목별 평가**
   - 평가 모델: **Qwen2.5** (예: `Qwen2.5-7B-Instruct`와 같이 실제 HF ID로 교체)
   - 항목별 평가 프롬프트(예: helpfulness, correctness, style 등)를 사용해 **1~10 점수** 산출
   - 학습 전 **베이스라인 모델**, LoRA 학습 후 **업데이트 모델** 각각에 대해 score 계산

2. **(선택) RAG 평가**
   - 주어진 정답 context 및 검색된 context 목록으로부터 **precision@k, recall@k** 계산 유틸 함수 제공

3. **기타 벤치마크 훅**
   - `mmlu`, `mt-bench` 등 외부 라이브러리를 이용한 평가를 쉽게 붙일 수 있도록 스켈레톤 코드 제공


In [23]:

import re
import torch
from typing import Dict, List, Any

# ==========================
# G-EVAL 설정
# ==========================

# ✏️ 실제 사용하는 Qwen3-8B 평가 모델 ID로 바꿔주세요.
# 예시: "Qwen2.5-Instruct" (실제 HF Hub 모델명 확인 후 교체)
QWEN_EVAL_MODEL_NAME = os.environ.get("QWEN_EVAL_MODEL_NAME", "Qwen/Qwen2.5-7B-Instruct")

print("Loading evaluator model:", QWEN_EVAL_MODEL_NAME)
from transformers import AutoTokenizer as EvalAutoTokenizer, AutoModelForCausalLM as EvalAutoModelForCausalLM

evaluator_tokenizer = EvalAutoTokenizer.from_pretrained(QWEN_EVAL_MODEL_NAME, use_fast=True)
evaluator_model = EvalAutoModelForCausalLM.from_pretrained(
    QWEN_EVAL_MODEL_NAME,
    torch_dtype="auto",
    device_map="auto"
)
evaluator_model.eval()

# 평가 항목별 프롬프트 템플릿 (원래 제공받은 프롬프트를 여기에 그대로 넣으면 됨)
# 각 value는 전체 시스템 prompt가 아니라, "평가 기준 설명 + 출력 형식"을 정의하는 템플릿입니다.
# {{user}}, {{reference}}, {{candidate}} 자리에 실제 텍스트를 넣습니다.
EVAL_CRITERIA_PROMPTS: Dict[str, str] = {
    "helpfulness": (
        "You are an expert judge for customer support responses.\n"
        "Evaluate how helpful the assistant's answer is to the user's question.\n"
        "User question:\n{{user}}\n\n"
        "Reference (ideal) answer:\n{{reference}}\n\n"
        "Candidate answer:\n{{candidate}}\n\n"
        "Give a score from 1 (very bad) to 10 (excellent).\n"
        "Output ONLY in the format: Score: <number>."
    ),
    "correctness": (
        "You are an expert judge for factual correctness.\n"
        "Evaluate how factually correct the candidate answer is compared to the reference.\n"
        "User question:\n{{user}}\n\n"
        "Reference (ground-truth) answer:\n{{reference}}\n\n"
        "Candidate answer:\n{{candidate}}\n\n"
        "Give a score from 1 (very incorrect) to 10 (fully correct).\n"
        "Output ONLY in the format: Score: <number>."
    ),
    "style": (
        "You are an expert judge for style and politeness.\n"
        "Evaluate if the candidate answer is polite, clear, and appropriate for customer support.\n"
        "User question:\n{{user}}\n\n"
        "Candidate answer:\n{{candidate}}\n\n"
        "Give a score from 1 (very poor) to 10 (excellent).\n"
        "Output ONLY in the format: Score: <number>."
    ),
    # 필요 시 다른 항목도 동일한 형식으로 추가
}

def build_g_eval_prompt(
    criterion: str,
    user: str,
    reference: str,
    candidate: str,
) -> str:
    """criterion에 해당하는 템플릿에 user/reference/candidate를 채워 넣어 평가 prompt 생성"""
    template = EVAL_CRITERIA_PROMPTS[criterion]
    prompt = (
        template
        .replace("{{user}}", user or "")
        .replace("{{reference}}", reference or "")
        .replace("{{candidate}}", candidate or "")
    )
    return prompt

def parse_score_from_text(text: str) -> float:
    """LLM 출력에서 'Score: X' 형태 또는 숫자만 포함된 경우를 robust하게 파싱."""
    # 우선 Score: X 패턴
    m = re.search(r"Score\s*[:=]\s*([0-9]+(?:\.[0-9]+)?)", text)
    if m:
        try:
            return float(m.group(1))
        except Exception:
            pass
    # 숫자 하나만 있는 경우 등 fallback
    m2 = re.search(r"([0-9]+(?:\.[0-9]+)?)", text)
    if m2:
        try:
            return float(m2.group(1))
        except Exception:
            pass
    return float("nan")

@torch.no_grad()
def run_g_eval_single(
    user: str,
    reference: str,
    candidate: str,
    criterion: str,
    max_new_tokens: int = 64,
) -> float:
    """단일 샘플에 대해 Qwen3-8B로 G-EVAL 점수 계산."""
    prompt = build_g_eval_prompt(criterion, user, reference, candidate)
    inputs = evaluator_tokenizer(prompt, return_tensors="pt").to(evaluator_model.device)
    outputs = evaluator_model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        temperature=0.0,
    )
    out_text = evaluator_tokenizer.decode(outputs[0], skip_special_tokens=True)
    score = parse_score_from_text(out_text)
    return score

def extract_eval_triplet_from_messages(example: Dict[str, Any]) -> Dict[str, str]:
    """dataset의 messages에서 (system, user, reference_answer)를 뽑아냄."""
    messages = example.get("messages", [])
    system = None
    last_user = None
    last_assistant = None
    for m in messages:
        role = m.get("role")
        content = m.get("content", "")
        if role == "system" and system is None:
            system = content
        elif role == "user":
            last_user = content
        elif role == "assistant":
            last_assistant = content
    if system is None:
        system = "You are a helpful, step-by-step customer support assistant."
    return {
        "system": system,
        "user": last_user or "",
        "reference": last_assistant or "",
    }

@torch.no_grad()
def generate_answer_from_model(
    model,
    tokenizer,
    system: str,
    user: str,
    max_new_tokens: int = 256,
    **gen_kwargs,
) -> str:
    """SFT에 사용한 chat_template 기반으로 답변 생성."""
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": user},
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    default_kwargs = dict(
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
    )
    default_kwargs.update(gen_kwargs)
    outputs = model.generate(**inputs, **default_kwargs)
    text = tokenizer.decode(outputs[0][inputs['input_ids'].shape[-1]:], skip_special_tokens=True)
    return text.strip()

# ==========================
# 베이스라인 / 업데이트 모델 로딩
# ==========================

# 🔹 베이스라인: LoRA 적용 전 원본 모델
print("Loading baseline model for evaluation:", MODEL_NAME)
baseline_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype="auto",
    device_map="auto"
)
baseline_model.eval()

# 🔹 업데이트 모델: 현재 노트북에서 LoRA로 fine-tune된 모델 (trainer 내부 모델 사용)
#   trainer.model이 LoRA 적용 모델이므로 그대로 사용
updated_model = trainer.model
updated_model.eval()

# ==========================
# G-EVAL 루프: eval_dataset 일부에 대해 점수 계산
# ==========================

def compute_geval_scores_for_model(
    model,
    tokenizer,
    eval_data,
    criteria: List[str],
    num_samples: int = 32,
    seed: int = SEED,
):
    """eval_data에서 일부 샘플을 뽑아 G-EVAL 점수 계산 후 항목별 평균 반환."""
    import random
    random.seed(seed)

    n = len(eval_data)
    idxs = list(range(n))
    random.shuffle(idxs)
    idxs = idxs[: min(num_samples, n)]

    scores = {c: [] for c in criteria}

    for idx in idxs:
        ex = eval_data[idx]
        triplet = extract_eval_triplet_from_messages(ex)
        system = triplet["system"]
        user = triplet["user"]
        reference = triplet["reference"]

        candidate = generate_answer_from_model(model, tokenizer, system, user)

        for c in criteria:
            s = run_g_eval_single(
                user=user,
                reference=reference,
                candidate=candidate,
                criterion=c,
            )
            scores[c].append(s)
    # 평균 계산
    avg_scores = {c: (sum(v)/len(v) if v else float("nan")) for c, v in scores.items()}
    return avg_scores, scores

CRITERIA = list(EVAL_CRITERIA_PROMPTS.keys())

print("Computing G-EVAL scores for baseline model...")
baseline_avg, baseline_all = compute_geval_scores_for_model(
    baseline_model,
    tokenizer,
    eval_dataset,
    criteria=CRITERIA,
    num_samples=16,   # 필요시 조절
)
print("[Baseline] average scores:", baseline_avg)

print("Computing G-EVAL scores for updated (LoRA) model...")
updated_avg, updated_all = compute_geval_scores_for_model(
    updated_model,
    tokenizer,
    eval_dataset,
    criteria=CRITERIA,
    num_samples=16,
)
print("[Updated] average scores:", updated_avg)


Loading evaluator model: Qwen/Qwen2.5-7B-Instruct


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Loading baseline model for evaluation: meta-llama/Llama-2-7b-hf


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Computing G-EVAL scores for baseline model...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[Baseline] average scores: {'helpfulness': 2.125, 'correctness': 2.375, 'style': 3.125}
Computing G-EVAL scores for updated (LoRA) model...
[Updated] average scores: {'helpfulness': 6.625, 'correctness': 6.1875, 'style': 7.5625}
